In [120]:
import pandas as pd
import geopy
import geopy.distance
from shapely.geometry import Polygon, LineString, Point 
from math import cos  
from math import sin  
from math import pi
from datetime import datetime, timedelta

In [121]:
met_data = pd.read_excel('Meteorological Data.xlsx')
sensor_data = pd.read_excel('Sensor Data.xlsx')

In [124]:
met_data_1 = met_data.copy()
met_data_2 = met_data.copy()

met_data_1['Date'] = met_data_1['Date'] + timedelta(hours=1)
met_data_2['Date'] = met_data_2['Date'] + timedelta(hours=2)

met_data = met_data.append([met_data_1, met_data_2]).sort_values('Date').reset_index(drop=True)

In [125]:
merged_data = sensor_data.merge(met_data[['Date', 'Wind Direction', 'Wind Speed (m/s)']].rename(columns={'Date': 'Date Time '}), on='Date Time ')
merged_data

,Chemical,Monitor,Date Time,Reading,Wind Direction,Wind Speed (m/s)
0,Methylosmolene,3,2016-04-01 00:00:00,2.683820,190.5,4.0
1,Methylosmolene,7,2016-04-01 00:00:00,2.630640,190.5,4.0
2,Chlorodinine,3,2016-04-01 00:00:00,1.259170,190.5,4.0
3,Chlorodinine,7,2016-04-01 00:00:00,0.943983,190.5,4.0
4,AGOC-3A,3,2016-04-01 00:00:00,0.722303,190.5,4.0
...,...,...,...,...,...,...
76068,AGOC-3A,1,2016-12-31 23:00:00,0.078266,161.0,3.5
76069,Methylosmolene,3,2016-12-31 23:00:00,0.062784,161.0,3.5
76070,AGOC-3A,2,2016-12-31 23:00:00,0.060814,161.0,3.5
76071,Appluimonia,7,2016-12-31 23:00:00,0.039729,161.0,3.5


In [127]:
for i, row in merged_data.iterrows():
    if row['Wind Direction'] <= 180:
        merged_data.loc[i, 'actual_direction'] = row['Wind Direction'] + 180
    else:
        merged_data.loc[i, 'actual_direction'] = row['Wind Direction'] - 180

In [128]:
sensor_loc_map = {1: [62,21],
                2: [66,35],
                3: [76,41],
                4: [88,45],
                5: [103,43],
                6: [102,22],
                7: [89,3],
                8: [74,7],
                9: [119,42]}

In [129]:
merged_data['Coordinates'] = merged_data['Monitor'].map(sensor_loc_map)
roadrunner = Point(89, 27)
kasios = Point(90, 21)
radiance = Point(109, 26)
indigo = Point(120, 22)

In [130]:
def factory_distances(x, y, angle):  

    angle = angle * pi / 180.0  
    line = LineString([(x, y), (x + 10000 * sin(angle), y + 10000 * cos(angle))]) 
    
    roadrunner_dist = roadrunner.distance(line)
    kasios_dist = kasios.distance(line)
    radiance_dist = radiance.distance(line)
    indigo_dist = indigo.distance(line)
    
    return roadrunner_dist, kasios_dist, radiance_dist, indigo_dist

In [131]:
for i, row in merged_data.iterrows():
    roadrunner_dist, kasios_dist, radiance_dist, indigo_dist = factory_distances(row['Coordinates'][0], row['Coordinates'][1], row['actual_direction'])
    merged_data.loc[i, 'roadrunner_dist'] = roadrunner_dist
    merged_data.loc[i, 'kasios_dist'] = kasios_dist
    merged_data.loc[i, 'radiance_dist'] = radiance_dist
    merged_data.loc[i, 'indigo_dist'] = indigo_dist
    
    l = [roadrunner_dist, kasios_dist, radiance_dist, indigo_dist]
    indices_order = [l.index(x) for x in sorted(l)]
    merged_data.loc[i, 'highest_contributor'] = indices_order[0]
    merged_data.loc[i, 'second_highest_contributor'] = indices_order[1]
    merged_data.loc[i, 'third_highest_contributor'] = indices_order[2]
    merged_data.loc[i, 'fourth_highest_contributor'] = indices_order[3]

In [132]:
factory_map = {0: 'Roadrunner Fitness Electronics',
                1: 'Kasios Office Furniture',
                2: 'Radiance ColourTek',
                3: 'Indigo Sol Boards',}

In [133]:
merged_data['highest_contributor'] = merged_data['highest_contributor'].map(factory_map)
merged_data['second_highest_contributor'] = merged_data['second_highest_contributor'].map(factory_map)
merged_data['third_highest_contributor'] = merged_data['third_highest_contributor'].map(factory_map)
merged_data['fourth_highest_contributor'] = merged_data['fourth_highest_contributor'].map(factory_map)

In [134]:
merged_data

,Chemical,Monitor,Date Time,Reading,Wind Direction,Wind Speed (m/s),actual_direction,Coordinates,roadrunner_dist,kasios_dist,radiance_dist,indigo_dist,highest_contributor,second_highest_contributor,third_highest_contributor,fourth_highest_contributor
0,Methylosmolene,3,2016-04-01 00:00:00,2.683820,190.5,4.0,10.5,"[76, 41]",19.104973,24.413111,36.249138,47.927028,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
1,Methylosmolene,7,2016-04-01 00:00:00,2.630640,190.5,4.0,10.5,"[89, 3]",4.373653,2.296985,15.473681,27.018427,Kasios Office Furniture,Roadrunner Fitness Electronics,Radiance ColourTek,Indigo Sol Boards
2,Chlorodinine,3,2016-04-01 00:00:00,1.259170,190.5,4.0,10.5,"[76, 41]",19.104973,24.413111,36.249138,47.927028,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
3,Chlorodinine,7,2016-04-01 00:00:00,0.943983,190.5,4.0,10.5,"[89, 3]",4.373653,2.296985,15.473681,27.018427,Kasios Office Furniture,Roadrunner Fitness Electronics,Radiance ColourTek,Indigo Sol Boards
4,AGOC-3A,3,2016-04-01 00:00:00,0.722303,190.5,4.0,10.5,"[76, 41]",19.104973,24.413111,36.249138,47.927028,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76068,AGOC-3A,1,2016-12-31 23:00:00,0.078266,161.0,3.5,341.0,"[62, 21]",27.658633,28.000000,47.265209,58.008620,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
76069,Methylosmolene,3,2016-12-31 23:00:00,0.062784,161.0,3.5,341.0,"[76, 41]",19.104973,24.413111,36.249138,47.927028,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
76070,AGOC-3A,2,2016-12-31 23:00:00,0.060814,161.0,3.5,341.0,"[66, 35]",24.351591,27.784888,43.931765,55.542776,Roadrunner Fitness Electronics,Kasios Office Furniture,Radiance ColourTek,Indigo Sol Boards
76071,Appluimonia,7,2016-12-31 23:00:00,0.039729,161.0,3.5,341.0,"[89, 3]",7.813636,6.805745,26.398439,35.496871,Kasios Office Furniture,Roadrunner Fitness Electronics,Radiance ColourTek,Indigo Sol Boards


In [138]:
merged_data.to_csv('complete_data.csv', index=False)